# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.sample(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
199,199,tutoia,-2.7619,-42.2744,25.73,84,24,6.27,BR,1726544989
285,285,alotau,-10.3069,150.4462,24.20,97,100,3.35,PG,1726545098
583,583,franceville,-1.6333,13.5836,21.50,96,81,0.53,GA,1726545462
547,547,orcopampa,-15.2661,-72.3417,8.45,48,18,0.72,PE,1726545418
142,142,hadibu,12.6500,54.0333,25.96,81,38,6.43,YE,1726544921
160,160,morondava,-20.2833,44.2833,24.04,78,8,0.78,MG,1726544942
590,590,tysmenytsya,48.9022,24.8448,12.04,84,99,0.92,UA,1726545471
509,509,tazovsky,67.4667,78.7000,5.69,89,92,2.94,RU,1726545373
19,19,waitangi,-43.9535,-176.5597,10.46,95,100,1.79,NZ,1726544775
110,110,vila franca do campo,37.7167,-25.4333,22.23,84,94,1.79,PT,1726544882


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM',
    frame_width = 1000, #forces an image size
    frame_height = 500, #forces an image size
    color = 'City', #sets different color for each city
    size = 'Humidity',
    scale = 0.8
)

# Display the map
humidity_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [71]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values; this drops rows with any null values
# print(city_data_df.isnull().sum()) -- perform a drop on Country based on is null print statement results

city_cleaned_df = city_data_df.dropna()

# Ideal temperature is between 75 and 60 degrees Farenheit with no more than 40% humidity
# 85 degrees Farenheit is 29.4 degrees Celsius; 60 degrees Farenheit is 15.6 degrees Celsius
ideal_city_df = city_cleaned_df[(city_cleaned_df['Max Temp'] >= 15.6) & (city_cleaned_df['Max Temp'] <= 29.4) & (city_cleaned_df['Humidity'] < 40)]

# Display sample data
ideal_city_df.sample(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
422,422,bafq,31.6128,55.4107,25.54,27,5,0.67,IR,1726545265
257,257,lamarque,-39.4230,-65.7021,19.26,29,99,9.75,AR,1726545062
381,381,ingeniero guillermo n. juarez,-23.9000,-61.8500,17.83,33,0,3.86,AR,1726545213
568,568,luau,-10.7073,22.2247,19.36,32,1,1.39,AO,1726545442
541,541,ulanhot,46.0833,122.0833,21.55,34,98,1.48,CN,1726545411
286,286,gilgit,35.9221,74.3087,21.23,30,100,1.50,PK,1726545100
504,504,ghadamis,30.1337,9.5007,27.30,30,41,1.35,LY,1726545366
300,300,barcelos,41.5388,-8.6151,19.53,36,9,4.81,PT,1726545116
228,228,geraldton,-28.7667,114.6000,28.23,21,0,7.72,AU,1726545027
491,491,ma'rib,15.4591,45.3253,28.00,16,56,1.49,YE,1726545349


### Step 3: Create a new DataFrame called `hotel_df`.

In [68]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.sample(20)


,City,Country,Lat,Lng,Humidity,Hotel Name
382,san rafael,AR,-34.6177,-68.3301,26,None
218,buka,UZ,40.8108,69.1986,34,None
257,lamarque,AR,-39.4230,-65.7021,29,None
245,al artawiyah,SA,26.5053,45.3444,21,None
156,kawambwa,ZM,-9.7915,29.0791,34,None
438,plastun,RU,44.7561,136.2939,39,None
404,baharly,TM,38.4362,57.4316,22,None
381,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,33,None
485,longjiang,CN,47.3303,123.1836,32,None
104,beyneu,KZ,45.3167,55.2000,25,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [69]:
# Set parameters to search for a hotel
category = 'accommodation.hotel'
radius = 10000
limit = 1
params = {"categories": category, "limit": limit, "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.sample(20)

Starting hotel search
kabalo - nearest hotel: No hotel found
leeton - nearest hotel: No hotel found
beyneu - nearest hotel: Нұр
gokwe - nearest hotel: No hotel found
kawambwa - nearest hotel: No hotel found
novyy urgal - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
chegdomyn - nearest hotel: Бурея
buka - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
zhanaozen - nearest hotel: No hotel found
al artawiyah - nearest hotel: No hotel found
lamarque - nearest hotel: No hotel found
novomykolayivka - nearest hotel: No hotel found
gilgit - nearest hotel: PTDC Motel
barcelos - nearest hotel: Hotel Dom Nuno
xiva - nearest hotel: B&B Orzu
tindouf - nearest hotel: محمد بوسبي
sampacho - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
san rafael - nearest hotel: Hotel Regional
broken hill - nearest hotel: Ibis Styles
korla - nearest hotel: Silve

,City,Country,Lat,Lng,Humidity,Hotel Name
307,tindouf,DZ,27.6711,-8.1474,38,محمد بوسبي
286,gilgit,PK,35.9221,74.3087,30,PTDC Motel
245,al artawiyah,SA,26.5053,45.3444,21,No hotel found
305,xiva,UZ,41.3783,60.3639,26,B&B Orzu
192,alice springs,AU,-23.7000,133.8833,12,Aurora Alice Springs
422,bafq,IR,31.6128,55.4107,27,هتل رستوران علمدار
257,lamarque,AR,-39.4230,-65.7021,29,No hotel found
461,nurota,UZ,40.5614,65.6886,34,Nurota
104,beyneu,KZ,45.3167,55.2000,25,Нұр
568,luau,AO,-10.7073,22.2247,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM',
    frame_width = 1000, #forces an image size
    frame_height = 500, #forces an image size
    color = 'City', #sets different color for each city
    size = 'Humidity',
    scale = 1.2,
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)